In [ ]:
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import init
from mxnet import nd
from mxnet import image
from mxnet.gluon.nn import Conv2D
from mxnet.gluon.nn import MaxPool2D
from mxnet import init
import mxnet as mx



In [ ]:
batch_size=64
num_epochs=10
learning_rate=1e-3

In [ ]:
# train_augs = [
#     image.HorizontalFlipAug(.5),
#     image.RandomCropAug((224,224))
#     ]

# test_augs = [
#     image.CenterCropAug((224,224))
#     ]

# def transform(data, label, augs):
#     data = data.astype('float32')
#     for aug in augs:
#         data = aug(data)
#     data = nd.transpose(data, (2,0,1))
#     return data, nd.array([label]).asscalar().astype('float32')

In [ ]:
def img_transform(data, label):
    data = data.transpose((2, 0, 1))
    return (data.astype('float32') / 255 - 0.5) / 0.5, label.astype('float32')

In [ ]:
dataset=gluon.data.vision.MNIST(transform=img_transform)
dataloader=gluon.data.DataLoader(dataset,batch_size,shuffle=True)

In [ ]:
class AutoEncoder(nn.HybridBlock):
    def __init__(self,**kwargs):
        super(AutoEncoder,self).__init__()
        with self.name_scope():
            self.encoder=nn.HybridSequential(prefix='encoder_')
            with self.encoder.name_scope():                               
                self.encoder.add(
                    Conv2D(16,3,strides=3,padding=1,activation='relu'),    # 16 channels , kernel_size=3
                    MaxPool2D(pool_size=(2,2)),
                    Conv2D(8,3,strides=2,padding=1,activation='relu'),
                    MaxPool2D(pool_size=(2,2)),
                )
            self.decoder=nn.HybridSequential(prefix='decoder_')
            with self.decoder.name_scope():
                self.decoder.add(
                    nn.Conv2DTranspose(16,3,strides=2,activation='relu'),
                    nn.Conv2DTranspose(8,5,strides=3,padding=1,activation='relu'),
                    nn.Conv2DTranspose(1,2,strides=2,padding=1,activation='tanh')
                )
                
    def forward(self,x):
        x=self.encoder(x)
        x=self.decoder(x)
        return x

In [ ]:
ctx=mx.gpu()
model=AutoEncoder()
model.hybridize()
model.collect_params().initialize(init.Xavier("gaussian"),ctx=ctx)

In [ ]:
model

In [ ]:
l2loss=gluon.loss.L2Loss()

In [ ]:
trainer=gluon.Trainer(model.collect_params(),'adam',{'learning_rate':learning_rate,"wd":1e-5})

In [ ]:
for epoch in range(num_epochs):
    for data in dataloader:
        image,_=data
        img=image.as_in_context(ctx)
        batch=image.shape[0]
        
        with mx.autograd.record():
            output=model(img)
            loss=l2loss(output,img)
            
        loss.backward()
        trainer.step(batch)
        
        print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, nd.mean(loss).asscalar()))
    

model.save_params('./con_autoencoder.params')

